In [1]:
from MUG_data_processing import load_MUG_data
from memory_profiler import memory_usage

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


cutFace algo


In [2]:
import cv2
import sys
import os
import numpy as np

#give grayscale image and 2D height and width of output and a facecascade
def cutFace(img,width,height,casc, database):

    try:
        if(database == 'MUG'):
            minSizeX = 100
            minSizeY = 100
        else:
            minSizeX = 100
            minSizeY = 100
        # Detect faces in the image
        faces = casc.detectMultiScale(
            img,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(minSizeX, minSizeY),
            flags = cv2.CASCADE_SCALE_IMAGE
        )
        if(len(faces)!=0):
            for (x, y, w, h) in faces:
                if w > h :
                    he = w//2
                    wi = w//2
                else :
                    he = h//2
                    wi = h//2

                x_m = (x+w//2)
                y_m = (y+h//2)
                
                cut_image = img[y_m-he:y_m+he,x_m-wi:x_m+wi]
                cut_image = cv2.resize(cut_image,(width,height), interpolation = cv2.INTER_AREA)
        
        return cut_image

    except Exception as e:
        print(e)

In [3]:
import copy
def load_formated_data(data,database, printData = 0, cascPath = "C:/Users/JeanBV/Documents/Thesis_Christiaan_Jean/Custom_Dexpression/haarcascade.xml"):

#     data = load_data(database)
#     print("test")
    #data[0] = X images
    #data[2] = Y emotion labels
    #data[3] = subject Ids
#     print("data:", data)
    images = data[0]
    emotion_labels = data[1]
    subjects = data[2]
    emotion_labels_new = []
    subjectID = []
    # cut faces from each images and resize (downsample) to input dimension
    X_gray = []
    X_test = []

    # Create the haar cascade
    faceCascade = cv2.CascadeClassifier(cascPath)

    
    for i in range(0,len(images)):
        cut_img = cutFace(images[i],224,224,faceCascade, database)
        #Be sure that cut_img is initialized
        if(cut_img is not None):
            X_gray = np.append(X_gray,cut_img)
            emotion_labels_new.append(emotion_labels[i])
            subjectID.append(subjects[i])
        
        if(i%50 ==0):
        	mem_usage = memory_usage(-1, interval=.2, timeout=1)
        	str = 'Maximum memory after cutting ' + repr(i) + " is " + repr( mem_usage)
        	print(str)
            
    #Resize to 224x224 image for input of neural networkCKE
    X_gray = X_gray.reshape([-1,224,224,1])

    # reformprint(at the emotionlabels to create a vector of 7 zeros only one of these elements is set to 1 indicating the emotion label
    labels = []

    for lab in emotion_labels_new:
        inst = np.zeros(7)
        inst[lab-1] = 1
        #print (inst)
        labels = np.append(labels,inst)

    print("size Y 2: " + repr(labels.shape))

    labels = labels.reshape([-1,7])

    print("size labels flatten: " + repr(len(labels)))
    print("size labels shape: " + repr(labels.shape))
    print("type labels: " + repr(type(labels)))

    outputData = [X_gray,labels,subjectID]

    print("ouputData", outputData)
    return outputData


    #data = load_formated_database(dataPath,0)

def create_formated_data(data,database, printData = 0, cascPath = "C:/Users/JeanBV/Documents/Thesis_Christiaan_Jean/Custom_Dexpression/haarcascade.xml" , databasePath = 'C:/Users/JeanBV/Documents/Thesis_Christiaan_Jean/Custom_Dexpression/'):
   
#     data = load_formated_data(database, printData, cascPath)
    
    os.chdir(databasePath)
    print("making dB")
  
    #Creating directories to save the npy files of the database
    database = database
    npy_folder = "output_npy"
    if not os.path.exists(npy_folder):
        os.makedirs(npy_folder)

    current_directory = os.path.abspath(npy_folder)
    subject_dir = os.path.join(current_directory, database) #The rule to get the folder INSIDE the folder
    if not os.path.exists(subject_dir):
        os.makedirs(subject_dir)
   
    X_data = data[0].astype('uint8')
    np.save(subject_dir+"\\"+database+'_X.npy',X_data)
    np.save(subject_dir+"\\"+database+'_Y.npy',data[1])
    np.save(subject_dir+"\\"+database+'_subjectIDs.npy',data[2])


In [4]:
# for Christiaan Vanbergen

database ='MUG'
cascPath = 'G:/Documenten/personal/school/MaNaMA_AI/thesis/implementation/dexpression/MMI_data_creation/Thesis_Christiaan_Jean/Custom_Dexpression/haarcascade.xml'
databasePath ='G:/Documenten/personal/school/MaNaMA_AI/thesis/implementation/dexpression/MMI_data_creation/Thesis_Christiaan_Jean/Custom_Dexpression/MUG'


In [5]:

mem_usage = memory_usage(-1, interval=.2, timeout=1)
str = 'Maximum memory before load ' + repr( mem_usage)
print(str)


Maximum memory before load [206.69140625, 206.69140625, 206.6953125, 206.6953125, 206.6953125]


In [7]:
# In[38]:
def main():
    data = load_MUG_data(database)

    mem_usage = memory_usage(-1, interval=.2, timeout=1)
    str = 'Maximum memory after loading data ' + repr( mem_usage)
    print(str)

    # In[43]:


    data = load_formated_data(data,database, 0, cascPath)

    mem_usage = memory_usage(-1, interval=.2, timeout=1)
    str = 'Maximum memory after formatting data ' + repr( mem_usage)
    print(str)

    # In[44]:


    create_formated_data(data,database,0,cascPath,databasePath)

    mem_usage = memory_usage(-1, interval=.2, timeout=1)
    str = 'Maximum memory creating formated data ' + repr( mem_usage)
    print(str)

In [8]:
# In[ ]:
if __name__ == '__main__':
    main()

length label_session_expression  69
['.\\002\\session0\\anger\\take000end\\', '.\\002\\session0\\anger\\take001\\', '.\\002\\session0\\anger\\take003\\', '.\\002\\session0\\anger\\take004\\', '.\\002\\session0\\disgust\\take000\\', '.\\002\\session0\\disgust\\take002\\', '.\\002\\session0\\disgust\\take003\\', '.\\002\\session0\\fear\\take000\\', '.\\002\\session0\\fear\\take001\\', '.\\002\\session0\\fear\\take002\\', '.\\002\\session0\\happiness\\take000\\', '.\\002\\session0\\happiness\\take001\\', '.\\002\\session0\\happiness\\take002\\', '.\\002\\session0\\neutral\\take000\\', '.\\002\\session0\\sadness\\take000\\', '.\\002\\session0\\sadness\\take001\\', '.\\002\\session0\\sadness\\take002\\', '.\\002\\session0\\sadness\\take003\\', '.\\002\\session0\\sadness\\take004\\', '.\\002\\session0\\sadness\\take006\\', '.\\002\\session0\\sadness\\take007\\', '.\\002\\session0\\surprise\\take000\\', '.\\002\\session0\\surprise\\take001\\', '.\\002\\session0\\surprise\\take002\\', '.\\002\

Maximum memory at label_expp 2300 is [207.01171875, 207.01171875, 207.01171875, 207.01171875, 207.01171875]
Maximum memory at label_expp 2300 is [207.01953125, 207.0234375, 207.0234375, 207.0234375, 207.0234375]
Maximum memory at label_expp 2300 is [207.0234375, 207.0234375, 207.0234375, 207.0234375, 207.0234375]
Maximum memory at label_expp 2350 is [207.0234375, 207.0234375, 207.0234375, 207.0234375, 207.0234375]
Maximum memory at label_expp 2400 is [207.0234375, 207.0234375, 207.0234375, 207.0234375, 207.0234375]
Maximum memory at label_expp 2450 is [207.03125, 207.03125, 207.03125, 207.03125, 207.03125]
Maximum memory at label_expp 2450 is [207.03125, 207.03125, 207.03125, 207.03125, 207.03125]
Maximum memory at label_expp 2500 is [207.03125, 207.03125, 207.03125, 207.03125, 207.03125]
Maximum memory at label_expp 2550 is [207.03125, 207.03125, 207.03125, 207.03125, 207.03125]
Maximum memory at label_expp 2600 is [207.04296875, 207.04296875, 207.04296875, 207.04296875, 207.04296875]

Maximum memory at label_expp 5450 is [207.2421875, 207.2421875, 207.2421875, 207.2421875, 207.2421875]
Maximum memory at label_expp 5500 is [207.2421875, 207.2421875, 207.2421875, 207.2421875, 207.2421875]
Maximum memory at label_expp 5550 is [207.2421875, 207.2421875, 207.2421875, 207.2421875, 207.2421875]
Maximum memory at label_expp 5550 is [207.2421875, 207.2421875, 207.2421875, 207.2421875, 207.2421875]
Maximum memory at label_expp 5600 is [207.2421875, 207.2421875, 207.2421875, 207.2421875, 207.2421875]
Maximum memory at label_expp 5650 is [207.2421875, 207.2421875, 207.2421875, 207.2421875, 207.2421875]
Maximum memory at label_expp 5700 is [207.2578125, 207.2578125, 207.2578125, 207.2578125, 207.2578125]
Maximum memory at label_expp 5700 is [207.2578125, 207.2578125, 207.2578125, 207.2578125, 207.2578125]
Maximum memory at label_expp 5750 is [207.2578125, 207.2578125, 207.2578125, 207.2578125, 207.2578125]
Maximum memory at label_expp 5750 is [207.2578125, 207.2578125, 207.25781

Maximum memory at label_expp 7900 is [207.3984375, 207.3984375, 207.3984375, 207.3984375, 207.3984375]
Maximum memory at label_expp 7950 is [207.3984375, 207.3984375, 207.3984375, 207.3984375, 207.3984375]
Maximum memory at label_expp 8000 is [207.40234375, 207.40234375, 207.40234375, 207.40234375, 207.40234375]
Maximum memory at label_expp 8050 is [207.40234375, 207.40234375, 207.40234375, 207.40234375, 207.40234375]
Maximum memory at label_expp 8100 is [207.40234375, 207.40234375, 207.40234375, 207.40234375, 207.40234375]
Maximum memory at label_expp 8150 is [207.40234375, 207.40234375, 207.40234375, 207.40234375, 207.40234375]
Maximum memory at label_expp 8200 is [207.40234375, 207.40234375, 207.40234375, 207.40234375, 207.40234375]
Maximum memory at label_expp 8250 is [207.40234375, 207.40234375, 207.40234375, 207.40234375, 207.40234375]
Maximum memory at label_expp 8300 is [207.40234375, 207.40234375, 207.40234375, 207.40234375, 207.40234375]
Maximum memory at label_expp 8300 is [

AttributeError: 'list' object has no attribute 'reshape'

In [10]:
i = 1
str = "hello " + repr(i+1)
print(str)

hello 2
